In [ ]:
from interfaces import *
from utilities import *
import numpy as np
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
#BSpline parameters
n = 4
p = 3
# Number of least square points
n_ls = 140

# Open knot vector
knots = np.zeros(n+2*(p+1))

knots = np.linspace(0,1,len(knots))
knots[0:p+1] = 0
knots[-p-1::] = 1
print len(knots), p
vsl = BsplineVectorSpace(p, knots)
print vsl.n_dofs
#BSpline parameters
n = 5
p = 3
# Number of least square points
n_ls = 140

# Open knot vector
knots = np.zeros(n+2*(p+1))

knots = np.linspace(0,1,len(knots))
knots[0:p+1] = 0
knots[-p-1::] = 1
print len(knots), p
vsl_long = BsplineVectorSpace(p, knots)
print vsl_long.n_dofs


In [ ]:
Guasto = np.load('GuastoPoints.npy')
start_frame = Guasto[0,0]
end_frame = Guasto[-1,0]
frames_per_stroke = int(end_frame-start_frame)+1
Guasto_CP=np.zeros([vsl.n_dofs+1,frames_per_stroke,3])
Guasto_CP_rep=np.zeros([vsl.n_dofs+1,frames_per_stroke,3])
points_per_frame = np.zeros(frames_per_stroke)
frame=start_frame
k=0
for i in range(Guasto.shape[0]):
    if(int(Guasto[i,0])!=frame):
        points_per_frame[k]=Guasto[i-1,1]
        frame=frame+1
        k=k+1
points_per_frame[-1]=Guasto[-1,1]
#print points_per_frame


In [ ]:
for i in range(frames_per_stroke):
    points = np.zeros([points_per_frame[i],2])
    if i == 0:
        start_index = 0
    else:
        start_index = int(np.sum(points_per_frame[:(i)]))
    points = Guasto[start_index:(start_index+points_per_frame[i]),2:]
    print points.shape
    #print points
    t = np.linspace(0,1,points_per_frame[i]) 
    Guasto_CP[1:,i,:2] = least_square_by_points(vsl, points, t)
    #print Guasto_CP[1:,i,:]
    
arky = ArcLengthParametrizer(vsl, Guasto_CP[1:,:,:], 3)
Guasto_CP_rep[1:,:,:]=arky.reparametrize()
Guasto_length=np.max(arky.lengths)
Julicher_length=6.15905483862
print Guasto_length,Julicher_length
plot(Guasto_CP_rep[1:,0,0]/Guasto_length*Julicher_length,Guasto_CP_rep[1:,0,1]/Guasto_length*Julicher_length,'o-',label='Original CPs')
Guasto_CP_rep[1:,:,:]=Guasto_CP_rep[1:,:,:]/Guasto_length*Julicher_length-np.array([0.1,0,0])
plot(Guasto_CP_rep[:,0,0],Guasto_CP_rep[:,0,1],'o-',label='Augmented CPs')
title('Control Point Addition')
legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=True, ncol=2)


In [ ]:
class Antenna(object):
    def __init__(self, cp, vs):
        self.cp = cp
        self.vs = vs
    
    def plot(self, t):
        s = linspace(0,1,50)
        c = self.vs.element(self.cp[:,t,:])
        plot(c(s)[0], c(s)[1])
        plot(self.cp[:,t,0], self.cp[:,t,1], '-or')

    def plot_curve(self, t):
        s = linspace(0,1,50)
        c = self.vs.element(self.cp[:,t,:])
        plot(c(s)[0], c(s)[1])
        #plot(self.cp[:,t,0], self.cp[:,t,1], '-or')
        


In [ ]:
t = np.linspace(0,1,100)
#for i in range(frames_per_stroke):
ant=Antenna(Guasto_CP_rep[1:,:,:],vsl)
ant.plot(1)



In [ ]:

for i in range(frames_per_stroke):
    ant.plot_curve(i)
    points = np.zeros([points_per_frame[i],2])
    if i == 0:
        start_index = 0
    else:
        start_index = int(np.sum(points_per_frame[:(i)]))
    points = Guasto[start_index:(start_index+points_per_frame[i]),2:]
    plot(points[:,0],points[:,1],'--')

#for i in range(frames_per_stroke):
    #ant.plot_curve(i)
title('NURBS Phlagellum Approximation')

In [ ]:
ant_long=Antenna(Guasto_CP_rep[:,:,:],vsl_long)
for i in range(frames_per_stroke):
    ant_long.plot_curve(i)

title('Guasto Stroke')

In [ ]:
ant.plot(0)
ant.plot(12)
ant.plot(11)
print Guasto_CP_rep.shape[1]

In [ ]:
Guasto_CP_rep[:,-1,:]=Guasto_CP_rep[:,0,:]

np.save('/Users/nicolagiuliani/giuliani-code/blender/CP_Guasto', Guasto_CP_rep[:,:-1,:])

In [ ]:
from interfaces import *
from utilities import *
import numpy as np
%pylab inline

CP_rep_arclength = np.zeros([vsl_long.n_dofs, frames_per_stroke, 3])
Guasto_CP_plus1 = np.zeros([vsl_long.n_dofs, frames_per_stroke, 3])
for i in range(frames_per_stroke):
    points = np.zeros([points_per_frame[i],2])
    if i == 0:
        start_index = 0
    else:
        start_index = int(np.sum(points_per_frame[:(i)]))
    points = Guasto[start_index:(start_index+points_per_frame[i]),2:]
    print points.shape
    #print points
    t = np.linspace(0,1,points_per_frame[i]) 
    Guasto_CP_plus1[:,i,:2] = least_square_by_points(vsl_long, points, t)
new_arky_long = ArcLengthParametrizer(vsl_long, Guasto_CP_plus1[:,:,:], 2, 1)
Guasto_CP_rep_arclength = new_arky_long.reparametrize()


In [ ]:
print Guasto_CP_rep_arclength.shape
ant_long = Antenna(Guasto_CP_rep_arclength, vsl_long)

for i in range(frames_per_stroke):
    ant_long.plot(i)

#xlim([-10,10])
#ylim([-10,10])
    
#print Guasto_CP_rep_arclength[0,:,:]
#print Guasto_CP_rep_arclength[1,:,:]
#print new_arky_long.rhsinit

In [ ]:
for i in range(frames_per_stroke):
    ant_long.plot_curve(i)
    points = np.zeros([points_per_frame[i],2])
    if i == 0:
        start_index = 0
    else:
        start_index = int(np.sum(points_per_frame[:(i)]))
    points = Guasto[start_index:(start_index+points_per_frame[i]),2:]
    #plot(points[:,0],points[:,1],'--')

title('Guasto Stroke')

In [ ]:
#BSpline parameters
n = 4
p = 3
# Number of least square points
n_ls = 140

# Open knot vector
knots = np.zeros(n+2*(p+1))

knots = np.linspace(0,1,len(knots))
knots[0:p+1] = 0
knots[-p-1::] = 1
print len(knots), p
vsl_try = BsplineVectorSpace(p, knots)
print vsl_try.n_dofs

Guasto_CP_try = np.zeros([vsl_try.n_dofs, frames_per_stroke, 3])
for i in range(frames_per_stroke):
    points = np.zeros([points_per_frame[i],2])
    if i == 0:
        start_index = 0
    else:
        start_index = int(np.sum(points_per_frame[:(i)]))
    points = Guasto[start_index:(start_index+points_per_frame[i]),2:]
    print points.shape
    #print points
    t = np.linspace(0,1,points_per_frame[i]) 
    Guasto_CP_try[:,i,:2] = least_square_by_points(vsl_try, points, t)
new_arky_try = ArcLengthParametrizer(vsl_try, Guasto_CP_try[:,:,:], 2, 1)
Guasto_CP_rep_try = new_arky_try.reparametrize()
Guasto_length=np.max(new_arky_try.lengths)
Julicher_length=6.15905483862
Guasto_CP_rep_try=Guasto_CP_rep_try/Guasto_length*Julicher_length
print Guasto_CP_rep_try.shape
ant_try = Antenna(Guasto_CP_rep_try,vsl_try)
for i in range(frames_per_stroke):
    ant_try.plot_curve(i)
    points = np.zeros([points_per_frame[i],2])
    if i == 0:
        start_index = 0
    else:
        start_index = int(np.sum(points_per_frame[:(i)]))
    points = Guasto[start_index:(start_index+points_per_frame[i]),2:]
    #plot(points[:,0],points[:,1],'--')



In [ ]:
Guasto_CP_rep_try[:,-1,:]=Guasto_CP_rep_try[:,0,:]
np.save('/Users/nicolagiuliani/giuliani-code/blender/CP_Guasto_rep', Guasto_CP_rep_try[:,:,:])
ant_try.plot(0)
ant_try.plot(12)
#ant_try.plot(12)

In [ ]:
Guasto_all=np.load('/Users/nicolagiuliani/giuliani-code/blender/Guasto_CPs_bis_rep.npy')
print Guasto_all.shape, vsl_try.n_dofs
ant_all=Antenna(Guasto_all[1:,:,:],vsl)
arky_all=ArcLengthParametrizer(vsl_try,Guasto_all[:,:,:],2,1)
Guasto_all_rep = np.zeros(Guasto_all.shape)
Guasto_all_rep[:,:,:] = arky_all.reparametrize()


In [ ]:
np.save('/Users/nicolagiuliani/giuliani-code/blender/CP_Guasto_all_rep', Guasto_all_rep[:,:,:])
ant_all_rep=Antenna(Guasto_all_rep[:,:,:],vsl_try)
for i in range(Guasto_all.shape[1]):
    ant_all_rep.plot(i)

In [ ]:
Guasto_all=np.load('Guasto_CPs_bis.npy')
print Guasto_all.shape
ant_all=Antenna(Guasto_all[1:,:,:],vsl)
ant_all.plot(12)
ant_all.plot(12*2)
arky_all=ArcLengthParametrizer(vsl,Guasto_all[1:,:,:],3)
Guasto_all_rep = np.zeros(Guasto_all.shape)
Guasto_all_rep[1:,:,:] = arky_all.reparametrize()
Guasto_all_rep[1:,:,:] = Guasto_all_rep[1:,:,:]-np.array([0.1,0,0])

In [ ]:
np.save('/Users/nicolagiuliani/giuliani-code/blender/CP_Guasto_all', Guasto_all_rep[:,:,:])
ant_all_rep=Antenna(Guasto_all_rep[1:,:,:],vsl)
for i in range(Guasto_all.shape[1]):
    ant_all_rep.plot(i)
    
    